# Notebook to develop rag evaluation methods

In [3]:
# Define API ENDPOINTS 
LLM_URL="http://10.103.251.104:8040/v1"
LLM_NAME="llama3"
MARQO_URL="http://10.103.251.104:8882"
# Old Marqo endpoint; version 1.5
# MARQO_URL="http://10.103.251.100:8882"


In [2]:
# Imports
import marqo
import re
import os
from langchain.text_splitter import (
    CharacterTextSplitter,  # need to install langchain
    NLTKTextSplitter,
    RecursiveCharacterTextSplitter,
)
from datasets import load_dataset
import pprint
import time
import random
import requests
from components import VectorStore, RagPipe, DatasetHelpers

In [ ]:
# Evaluation for a single rag element
example_entry = {
    "question": "What is the capital of France?",
    "answer": "Paris",
    "contexts": [
        "Paris is the capital of France and a major European city.",
        "Paris is located on the River Seine in northern France.",
        "Marseille is known for its art, culture, and landmarks."
    ],
    "context_ids": ["1", "2", "3"],
    "ground_truth": "The capital of France is Paris."
}

print(example_entry)



In [4]:
# Run eval in a few lines

# Load the dataset
dataset = DatasetHelpers()
corpus_list, queries, ground_truths = dataset.loadMiniWiki()

# Load the VectorStore
documentDB = VectorStore(MARQO_URL) # Connect to marqo client via python API
print(documentDB.getIndexes()) # Print all indexes
documentDB.connectIndex("miniwikiindex") # Connect to the miniwikiindex

# Load the RagPipe
pipe = RagPipe()
pipe.connectVectorStore(documentDB)
pipe.connectLLM(LLM_URL, LLM_NAME)

# Run the rag pipeline and ingest
pipe.run(queries,ground_truths, corpus_list,newIngest=False,maxDocs=10,maxQueries=3)




Loading MiniWiki dataset
[{'indexName': 'miniwikiindex'}, {'indexName': 'ait-qm'}]
Index connected: miniwikiindex 
 Language model URL: http://10.103.251.104:8040/v1
 Language model connected: llama3
Using already indexed documents
Index Stats:  {'numberOfDocuments': 10, 'numberOfVectors': 15, 'backend': {'memoryUsedPercentage': 0.08814318706999999, 'storageUsedPercentage': 30.90243340992}}
Start answering queries. Please wait. 
Current Question: Was Abraham Lincoln the sixteenth President of the United States?
Sending query to OpenAI endpoint: http://10.103.251.104:8040/v1/chat/completions
Received response...
Current Question: Did Lincoln sign the National Banking Act of 1863?
Sending query to OpenAI endpoint: http://10.103.251.104:8040/v1/chat/completions
Received response...
Current Question: Did his mother die of pneumonia?
Sending query to OpenAI endpoint: http://10.103.251.104:8040/v1/chat/completions
Received response...


In [7]:
# Evaluate the rag pipeline
pipe.eval(method="context_relevance")

Answer: A very easy one!

Yes, Abraham Lincoln was indeed the 16th President of the United States! He served from March 1861 until his assassination in April 1865.
Ground truth: yes


/home/simon/master_project/software/venv_koenigsi/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



 Similarity Score = tensor([[0.1328]]) 
Answer: No, Abraham Lincoln did not sign the National Banking Act of 1863.

The National Banking Act was actually signed into law by President Andrew Johnson on February 25, 1865. This act established a national banking system in the United States and created a new type of bank charter called a "national bank." The act also required national banks to invest a certain percentage of their capital in U.S. government securities.

Abraham Lincoln was assassinated on April 14, 1865, and died the next morning, so he did not have an opportunity to sign this legislation into law.
Ground truth: yes

 Similarity Score = tensor([[0.1200]]) 
Answer: I apologize, but this prompt doesn't seem to be related to the previous conversation about Uruguay. Could you please provide more context or clarify what you would like to know about someone's mother passing away from pneumonia? I'll do my best to help!
Ground truth: no

 Similarity Score = tensor([[0.0170]]) 


[tensor([[0.1328]]), tensor([[0.1200]]), tensor([[0.0170]])]